<a href="https://colab.research.google.com/github/eric-huang5497/Math-152/blob/main/Exploration_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exploration of the 3n+1 Conjecture
##By Aaron, Eric Huang, Brian Milne, Ethan Alon, Yuelian Li. 
###Introduction



In [ ]:
def efficient_collatz(n):
  '''
  The modified_collatz function below is pretty inefficient because there are lots of intermediate steps for each calculation.
  This is about as efficient as I think the usual collatz function can get, but if someone has a better idea, feel free to put it in.
  '''
  while n > 1:
    if n %2 == 0:
      n = n//2
    else:
      n = 3*n + 1
  return True 

##Analyzing Negative and Non-natural Numbers## 

##Changing the rules

Small changes to the two rules that describe the Collatz function can result in dramatic changes to the overall behavior of the function. For example, if we replace $n$ by $3n + 5$ when $n$ is odd, we find that there appear to be six distinct loops that all positive integers eventually reach, rather than the single loop we observe with the usual Collatz map. Instead of writing a brand new function for each new ruleset we wished to investigate, we wrote a function that could take in a starting value and a list of rules, and output the corresponding sequence.

The `modified_collatz` function, and the other functions defined below, take a parameter called `ruleset`, which is expected to be a list of two-tuples $[(a_0, b_0), \dots, (a_{k-1},b_{k-1})]$. Given $n$, it replaces $n$ by $a_i \cdot n + b_i$, where $i$ is the value of $n$ modulo $k$. For example, the behavior of the usual Collatz function is captured by the list $[(1/2,0), (3,1)]$.

In [17]:
def modified_collatz(n, ruleset = [(1/2,0), (3,1)], iterations = 200):
  '''
  The default value for ruleset is the list [(1/2,0), (3,1)], which gives the usual collatz function. 
  By default, len(ruleset)=2, so if n%2 == 0, we multiply n by 1/2 and add 0, and if n%2 == 1, we multiply n by 3 and add 1.

  '''
  arity = len(ruleset) #The modulus it checks n against to determine which rule to apply
  for i in range(iterations): 
    print(n)
    rem = n%arity #Picks out the appropriate rule from ruleset, based on the value of n modulo arity
    n = int(ruleset[rem][0]*n + ruleset[rem][1]) 

def update(n,ruleset):
  '''
  Given a starting value and a list of rules, returns the next number in the sequence
  '''
  arity = len(ruleset)
  rem = n%arity
  return int(ruleset[rem][0]*n + ruleset[rem][1])

modified_collatz(31, iterations = 110)




31
94
47
142
71
214
107
322
161
484
242
121
364
182
91
274
137
412
206
103
310
155
466
233
700
350
175
526
263
790
395
1186
593
1780
890
445
1336
668
334
167
502
251
754
377
1132
566
283
850
425
1276
638
319
958
479
1438
719
2158
1079
3238
1619
4858
2429
7288
3644
1822
911
2734
1367
4102
2051
6154
3077
9232
4616
2308
1154
577
1732
866
433
1300
650
325
976
488
244
122
61
184
92
46
23
70
35
106
53
160
80
40
20
10
5
16
8
4
2
1
4
2
1


Now that we have a system in place which makes it simple to alter the rules of the Collatz function and observe how the behavior changes, we can search for rulesets which have particularly interesting behaviors. As what constitutes an "interesting behavior" is subjective, we focused on investigating the loops that arise with a given ruleset. Here, if $f$ is a modified Collatz function, we define a "loop" to be the sequence of values obtained starting with $n$, whenever $f^k(n) = n$ for some $k > 1$. We make the convention that we present loops so that the smallest element of the loop is the first listed.

The Collatz conjecture states that no Collatz sequence diverges to infinity, and there is only a single loop which all positive integers eventually enter. Understanding how changes to the ruleset change the number and size of the loops produced would shed some light on why the usual Collatz sequence only has a single loop. The functions below are meant to aid in collecting statistics about the loops coming from different rulesets.



In [110]:
def modified_collatz_loop_finder(n,ruleset,max_iterations = 1000):
  '''
  If the collatz sequence beginning with n results in a loop where n is the smallest element, this will return the loop found.
  If the sequence enters a loop which has a smaller element than n, this will return nothing.
  If it is inconclusive if the sequence enters a loop, this prints a message stating so.
  '''
  elements = [n] #The list of elements reached starting from n
  tracker = update(n,ruleset) #Tracks the latest value in the sequence
  iterations = 0
  while tracker not in elements and tracker > n and iterations < max_iterations: 
    #Terminates if one of three conditions are met:
    #1. The sequence repeats a value
    #2. The sequence decreases below the starting value n
    #3. The length of the sequence exceeds a maximum size
    #Each iteration updates tracker and appends it to the list of elements
    elements.append(tracker)
    iterations += 1
    tracker = update(tracker,ruleset)
  if iterations == max_iterations: #This means that it is inconclusive if the sequence enters a loop or not.
    print(ruleset)
    print("Starting with {} produces either a very long sequence before looping, or an infinite ascending chain".format(n))
  elif tracker == n: #This means that n is the smallest element of a loop.
    return len(elements), elements

def num_loops_finder(ruleset, max_checked = 1000, max_iterations = 1000): 
  '''
  Finds all loops that have a starting value less than max_checked and length less than max_iterations, and returns the number of loops found. 
  '''
  num_loops = 0
  for i in range(1,max_checked):
    if type(modified_collatz_loop_finder(i,ruleset,max_iterations)) == tuple:
      num_loops += 1
  return num_loops

def longest_loop_length(ruleset, max_checked = 1000,max_iterations = 1000):
  '''
  Finds all loops that have a starting value less than max_checked and length less than max_iterations, and returns the length of the longest loop found.
  '''
  longest_length = 0
  for i in range(1,max_checked):
    if type(modified_collatz_loop_finder(i,ruleset,max_iterations)) == tuple:
      longest_length = max(longest_length, modified_collatz_loop_finder(i,ruleset,max_iterations)[0])
  return longest_length

Below, we consider a modification to the rules of the Collatz function, where we instead choose what to do based on the value of $n$ mod 3. Specifically, we have
$$f(n) = \begin{cases} n/3 \text{ if } n \equiv 0 \pmod{3} \\ 4n + 2 \text{ if } n \equiv 1 \pmod{3} \\ 4n+1 \text{ if } n\equiv 2 \pmod{3} \end{cases} $$

Below, we collect some statistice about loops found by starting under 10000, and print out all the loops found.

In [109]:
rules_list = [(1/3,0),(4,2), (4,1)]
print("number of loops under 10000 is", num_loops_finder(rules_list, 10000, 400))
print("longest loop length under 10000 is", longest_loop_length(rules_list, 10000, 400))

for i in range(1,10000):
  if modified_collatz_loop_finder(i,rules_list,400) != None:
    print(modified_collatz_loop_finder(i,rules_list,400))

number of loops under 10000 is 2
longest loop length under 10000 is 16
(5, [1, 6, 2, 9, 3])
(16, [7, 30, 10, 42, 14, 57, 19, 78, 26, 105, 35, 141, 47, 189, 63, 21])


For a more involved example, we can search for interesting loop behavior over a collection of maps of the form 
$$C_k(n) = \begin{cases} n/2 \text{ if } n \equiv 0 \pmod{2} \\ 3n+k \text{ if } n \equiv 1 \pmod{2} \end{cases} $$
where $k$ is an odd integer. Specifically, suppose we are looking for rulesets which result in one of three things:
1. Exactly one loop
2. More than 30 distinct loops
3. A loop with length greater than 400

We can find values of $k$ satisfying the above fairly easily using the functions defined earlier. In all cases, we check all starting values up to 10000, so it is possible that for a given $k$, there exist more loops which have a starting value greater than 10000. We first consider the first interesting critereon. 

In [126]:
print("These have exactly one loop:")
for k in range(1,1000,2):
  if num_loops_finder([(1/2,0), (3,k)], 10000, 1000) == 1:
    print("k = {} has exactly one loop".format(k))

These have exactly one loop:
k = 1 has exactly one loop
k = 3 has exactly one loop
k = 9 has exactly one loop
k = 27 has exactly one loop
k = 81 has exactly one loop
k = 243 has exactly one loop
k = 729 has exactly one loop


The values of $k$ we found which have exactly one loop suggest that it is precisely the powers of 3 which result in a unique loop. Some thought reveals why powers of 3 behave in the same way: the assignment $n = 3n + 3^m$ will increases the highest power of $3$ which divides $n$, until $3^m \mid n$. Then, if $n == 3^mj$, we have $n = 3^{m+1}j + 3^m == 3^m(3j + 1)$, so we simply have the regular Collatz function, with an extra factor of $3^m$. Indeed, we can see below that all values of $k$ of the form $5\cdot 3^i$ appear to have 6 distinct loops.

In [131]:
for i in range(5):
  print("5*3**{} has {} loops".format(i,num_loops_finder([(1/2,0), (3,5*3**i)], 100000, 1000)))

5*3**0 has 6 loops
5*3**1 has 6 loops
5*3**2 has 6 loops
5*3**3 has 6 loops
5*3**4 has 6 loops


Now, we search for values of $k$ which have many loops, or particularly long loops.

In [111]:
print("These have lots of loops:")
for k in range(1,1000,2):
  if num_loops_finder([(1/2,0), (3,k)], 10000, 1000) > 30:
    print("k = {} has {} loops".format(k,num_loops_finder([(1/2,0), (3,k)], 10000, 1000)))

These have lots of loops:
k = 499 has 49 loops
k = 781 has 36 loops
k = 893 has 32 loops


In [112]:
print("These have long loops:")
for k in range(1,1000,2):
  if longest_loop_length([(1/2,0), (3,k)], 10000, 1000) > 400:
    print("k = {} has a loop of length {}".format(k,longest_loop_length([(1/2,0), (3,k)], 10000, 1000)))

These have long loops:
k = 509 has a loop of length 404
k = 563 has a loop of length 641
k = 751 has a loop of length 429
k = 853 has a loop of length 451
k = 869 has a loop of length 446
k = 947 has a loop of length 421
k = 961 has a loop of length 405
k = 977 has a loop of length 412


Now that we have found some values of $k$ which have many loops, or particularly long loops, we can take a closer look by listing all the loops found. This also allows us to search up to a much higher maximum starting value of 1,000,000, which allows us to be more confident that we have found all the loops there are.

In [123]:
print('Loops for k = 499:')

for i in range(1,1000000):
  if modified_collatz_loop_finder(i,[(1/2,0), (3,499)],1000) != None:
    print(modified_collatz_loop_finder(i,[(1/2,0), (3,499)],1000))

Loops for k = 499:
(66, [1, 502, 251, 1252, 626, 313, 1438, 719, 2656, 1328, 664, 332, 166, 83, 748, 374, 187, 1060, 530, 265, 1294, 647, 2440, 1220, 610, 305, 1414, 707, 2620, 1310, 655, 2464, 1232, 616, 308, 154, 77, 730, 365, 1594, 797, 2890, 1445, 4834, 2417, 7750, 3875, 12124, 6062, 3031, 9592, 4796, 2398, 1199, 4096, 2048, 1024, 512, 256, 128, 64, 32, 16, 8, 4, 2])
(66, [23, 568, 284, 142, 71, 712, 356, 178, 89, 766, 383, 1648, 824, 412, 206, 103, 808, 404, 202, 101, 802, 401, 1702, 851, 3052, 1526, 763, 2788, 1394, 697, 2590, 1295, 4384, 2192, 1096, 548, 274, 137, 910, 455, 1864, 932, 466, 233, 1198, 599, 2296, 1148, 574, 287, 1360, 680, 340, 170, 85, 754, 377, 1630, 815, 2944, 1472, 736, 368, 184, 92, 46])
(300, [139, 916, 458, 229, 1186, 593, 2278, 1139, 3916, 1958, 979, 3436, 1718, 859, 3076, 1538, 769, 2806, 1403, 4708, 2354, 1177, 4030, 2015, 6544, 3272, 1636, 818, 409, 1726, 863, 3088, 1544, 772, 386, 193, 1078, 539, 2116, 1058, 529, 2086, 1043, 3628, 1814, 907, 3220, 1610

In [124]:
print('Loops for k = 781:')

for i in range(1,1000000):
  if modified_collatz_loop_finder(i,[(1/2,0), (3,781)],1000) != None:
    print(modified_collatz_loop_finder(i,[(1/2,0), (3,781)],1000))

Loops for k = 781:
(60, [29, 868, 434, 217, 1432, 716, 358, 179, 1318, 659, 2758, 1379, 4918, 2459, 8158, 4079, 13018, 6509, 20308, 10154, 5077, 16012, 8006, 4003, 12790, 6395, 19966, 9983, 30730, 15365, 46876, 23438, 11719, 35938, 17969, 54688, 27344, 13672, 6836, 3418, 1709, 5908, 2954, 1477, 5212, 2606, 1303, 4690, 2345, 7816, 3908, 1954, 977, 3712, 1856, 928, 464, 232, 116, 58])
(8, [71, 994, 497, 2272, 1136, 568, 284, 142])
(30, [83, 1030, 515, 2326, 1163, 4270, 2135, 7186, 3593, 11560, 5780, 2890, 1445, 5116, 2558, 1279, 4618, 2309, 7708, 3854, 1927, 6562, 3281, 10624, 5312, 2656, 1328, 664, 332, 166])
(45, [107, 1102, 551, 2434, 1217, 4432, 2216, 1108, 554, 277, 1612, 806, 403, 1990, 995, 3766, 1883, 6430, 3215, 10426, 5213, 16420, 8210, 4105, 13096, 6548, 3274, 1637, 5692, 2846, 1423, 5050, 2525, 8356, 4178, 2089, 7048, 3524, 1762, 881, 3424, 1712, 856, 428, 214])
(45, [119, 1138, 569, 2488, 1244, 622, 311, 1714, 857, 3352, 1676, 838, 419, 2038, 1019, 3838, 1919, 6538, 3269, 10

In [128]:
print('Loops for k = 9:')

for i in range(1,1000000):
  if modified_collatz_loop_finder(i,[(1/2,0), (3,9)],1000) != None:
    print(modified_collatz_loop_finder(i,[(1/2,0), (3,9)],1000))

Loops for k = 9:
(3, [9, 36, 18])


For the values of $k$ which have a large number of loops, we can immediately make an observation: almost all the loops for $k = 499$ have length 26 or a multiple of 26, and almost all the loops for $k = 781$ have length 15 or a multiple of 15. 